In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [12]:
service = Service(excutable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.maximize_window()
driver.get('https://www.linkedin.com/')


wait = WebDriverWait(driver, 10)

In [13]:
# Function to smoothly scroll to the bottom of the page
def smooth_scroll_to_bottom():
    # Get the initial page height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # Scroll down by a small increment
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
        
        # Wait for new content to load
        time.sleep(0.5)  # Adjust the sleep time as needed
        
        # Get the new page height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # If no new content is loaded, stop scrolling
        if new_height == last_height:
            break
        
        # Update the last height
        last_height = new_height


In [14]:

# Navigate to the jobs page 
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "nav")))
driver.find_element(By.XPATH, "//a[@href='https://www.linkedin.com/jobs/search?trk=guest_homepage-basic_guest_nav_menu_jobs']").click()

# close if there is any popup modal 
try:
    driver.find_element(By.CLASS_NAME, "modal__dismiss").click()
except Exception as e:
    print(f"Model {e}")
    


In [15]:
# Scroll to load all content
smooth_scroll_to_bottom()
# job_search_results = driver.find_element(By.CLASS_NAME, "jobs-search__results-list")
jobs = driver.find_elements(By.CLASS_NAME, "job-search-card")
print(len(jobs))

59


In [17]:
# locate the list of jobs 
jobs = driver.find_elements(By.CLASS_NAME, "job-search-card")
print(len(jobs))
jobs_fetched = []
number_of_jobs_to_be_scrap = 10
try:
    # loop through each job
    for job_counter in range(number_of_jobs_to_be_scrap):  
        # re-locate the list of jobs of to avoid stale elements
        jobs = driver.find_elements(By.CLASS_NAME, "job-search-card")
        number_of_jobs_loaded = len(jobs)
        
        # click on the current job 
        job = jobs[job_counter]
        job.find_element(By.CLASS_NAME, "base-card__full-link").click()
    
        # wait for the detailed page to load 
        time.sleep(1)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "two-pane-serp-page__detail-view")))
    
        # Extract detailed information 
        job_title = driver.find_element(By.CLASS_NAME, "top-card-layout__title").text
        company_name = driver.find_element(By.CLASS_NAME, "topcard__org-name-link").text
        job_location = driver.find_element(By.CLASS_NAME, "topcard__flavor--bullet").text
        job_description= driver.find_element(By.CLASS_NAME, "decorated-job-posting__details").get_attribute("innerText")

        job_detail = {
            "Title": job_title,
            "Company": company_name,
            "Location": job_location,
            "Description": job_description
        }
        jobs_fetched.append(job_detail)
        
        # print("-" * 100)
        # print(f'Job {job_counter+1}')
        # print("-" * 100)
        # print(f"Title: {job_title}")
        # print(f"Company: {company_name}")
        # print(f"Location: {job_location}")
        # print(f"Description: {job_description}")
        # print("-" * 100)
    
        # # Go back to the list of jobs 
        driver.back
        
        try:
            if job_counter == number_of_jobs_loaded-1:
                time.sleep(2)
                try:
                    see_more_jobs = driver.find_element(By.XPATH, "//button[@aria-label='See more jobs']")
                    if see_more_jobs:
                        see_more_jobs.click()
                        time.sleep(2)
                except:
                    pass
                    
                finally:
                    # smooth_scroll_to_bottom()
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    
        except:
            pass
    
        try:
            see_more_jobs = driver.find_element(By.XPATH, "//button[@aria-label='See more jobs']")
            if see_more_jobs:
                time.sleep(2)
                # see_more_jobs.click()
                driver.execute_script("arguments[0].click();", see_more_jobs)
                # jobs = driver.find_elements(By.CLASS_NAME, "job-search-card")
    
        except:
            pass
    
        # # wait for the list to load again 
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "job-search-card")))
    
except Exception as e:
                print(f'Scraped {job_counter} jobs, there are no more jobs in the search results: {e}')

finally: 
    print(f'Scraped {len(jobs_fetched)} jobs sucessfully')
    # close the brower
    driver.quit()


MaxRetryError: HTTPConnectionPool(host='localhost', port=56817): Max retries exceeded with url: /session/4c8bcb7d1f2dd24d4e015853c256c7e5/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7891449513a0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [8]:
len(jobs_fetched)

1

In [9]:
company_url = driver.find_element(By.XPATH, "//a[@href]")

In [10]:
company_url.get_attribute("href")

'https://www.linkedin.com/jobs/search?trk=guest_homepage-basic_guest_nav_menu_jobs&currentJobId=4196835390&position=1&pageNum=0#main-content'